In [52]:
import sys, os, glob
from glob import glob
import boto3
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import folium
import datetime
from os.path import join

In [33]:
ais_dir = join(os.path.expanduser("~"), 'data', 'AIS')

In [5]:
aws_bucket = "wbgdecinternal-ntl"
path = "Andres_Temp/AIS"

In [6]:
client = boto3.client('s3')

In [7]:
file_list = client.list_objects_v2(Bucket=aws_bucket, Prefix=path, MaxKeys=5000)
bucket_files = [os.path.join("s3://", aws_bucket, content['Key']) for content in file_list['Contents']]
bucket_files

['s3://wbgdecinternal-ntl/Andres_Temp/AIS/',
 's3://wbgdecinternal-ntl/Andres_Temp/AIS/AIS_MMSI_2019-01-01_2019-06-30.csv',
 's3://wbgdecinternal-ntl/Andres_Temp/AIS/books.csv',
 's3://wbgdecinternal-ntl/Andres_Temp/AIS/port_calls_latakia.csv',
 's3://wbgdecinternal-ntl/Andres_Temp/AIS/port_calls_test.csv']

In [8]:
df = pd.read_csv(bucket_files[1])

In [9]:
df.columns

Index(['message_type', 'mmsi', 'dt_insert_utc', 'longitude', 'latitude', 'imo',
       'vessel_name', 'callsign', 'vessel_type', 'vessel_type_code',
       'vessel_type_cargo', 'vessel_class', 'length', 'width', 'flag_country',
       'flag_code', 'destination', 'eta', 'draught', 'sog', 'cog', 'rot',
       'heading', 'nav_status', 'nav_status_code', 'source', 'dt_pos_utc',
       'dt_static_utc', 'vessel_type_main', 'vessel_type_sub', 'eeid',
       'source_filename', 'H3index_0', 'H3_int_index_0', 'H3_int_index_1',
       'H3_int_index_2', 'H3_int_index_3', 'H3_int_index_4', 'H3_int_index_5',
       'H3_int_index_6', 'H3_int_index_7', 'H3_int_index_8', 'H3_int_index_9',
       'H3_int_index_10', 'H3_int_index_11', 'H3_int_index_12',
       'H3_int_index_13', 'H3_int_index_14', 'H3_int_index_15'],
      dtype='object')

In [10]:
df.loc[:, "geometry"] = df.apply(lambda x: Point(x.longitude, x.latitude), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.crs = 'EPSG:4326'

/home/wb514197/.conda/envs/ee/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [25]:
data = gdf.copy()

In [16]:
data[['Date','Time']] = data.dt_pos_utc.str.split(' ',expand=True)

In [17]:
data['hour'] = pd.to_datetime(data['Time'], format='%H:%M:%S',errors = 'coerce').dt.hour
data['dtg'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])

In [67]:
month = data.loc[data.dtg<"2019-03-01"].copy()

In [68]:
gdf = month.copy()

In [65]:
ports = gpd.read_file(join(ais_dir, "Port_mapping.shp"))

In [57]:
ports.to_csv(join(ais_dir, "Port_mapping.csv"))

In [59]:
# ports

In [48]:
gdf_ports = gdf.loc[gdf.intersects(ports.unary_union)].copy()

In [56]:
len(gdf_ports), len(gdf)

(1098, 3138)

In [69]:
m = folium.Map(location=(gdf.iloc[0].latitude, gdf.iloc[0].longitude), zoom_start=13)
# geo_j = gdf.geometry.to_json()
# geo_j = folium.GeoJson(data=geo_j,
#                        style_function=lambda x: {'fillColor': 'orange'})
# geo_j.add_to(m)
pol_m = gdf.geometry.to_json()
pol = folium.GeoJson(pol_m,
                    control = True,
                    marker = folium.CircleMarker(
                        radius = 3, # Radius in metres
                        weight = 1, #outline weight
                        fill_color = 'teal', 
                        fill_opacity = 1,
                        color = 'white'
                    ),
                    # tooltip = folium.GeoJsonTooltip(fields = ['feeder'],
                    #                                 aliases=['Feeder: '],
                    #                                 style = ("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
                    #                                 sticky = True
                    # )
                    ).add_to(m)
por_m = ports.geometry.to_json()
pol2= folium.GeoJson(por_m,
                    control = True,
#                     marker = folium.CircleMarker(
#                         radius = 3, # Radius in metres
#                         weight = 1, #outline weight
#                         fill_color = 'teal', 
#                         fill_opacity = 1,
#                         color = 'white'
#                     )
                    ).add_to(m)

a = folium.LayerControl().add_to(m)
m